# Modulo 2
En este notebook se realiza el calculo de las emisiones.
Se utilizan los VKT desde el programa de Nicolásy se multiplican por los factores de emisión obtenidos por COPERT5.5

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

In [18]:
df = pd.read_excel("Modulo 2/VKT.xlsx", index_col=None)
EF = pd.read_excel("EF.xlsx", index_col=None)

In [67]:
df_2 = df.iloc[:,2:]
df_2.drop("2013", inplace=True, axis=1)

In [106]:
#verificar los años de los VKT y los años de las velocidades, deben calzar
def merger(df1,df2):
    dfm = df1.merge(df2, left_on=["Región", "Ámbito", "Categoría", "Motorizacion", "Norma"], right_on=["Region", "Ambito", "Modo", "Motorizacion", "Norma"], how="left")
    dfm = dfm.fillna(0)
    df_f = pd.DataFrame(dfm.loc[:,"2014_x":"2050"].values*dfm.loc[:,"2014_y":2050].values, columns=dfm.iloc[:,47:84].columns, index=dfm.index)
    df_f = pd.concat([dfm.iloc[:,:6],df_f],axis=1)
    df_f["Unidad"]="[gr/año]"
    return df_f

In [107]:
EF_CO = EF.iloc[:,1:-10][EF["Pollutant"]=="CO"]
EF_CH4 = EF.iloc[:,1:-10][EF["Pollutant"]=="CH4"]
EF_NOx = EF.iloc[:,1:-10][EF["Pollutant"]=="NOx"]
EF_CO2 = EF.iloc[:,1:-10][EF["Pollutant"]=="CO2"]
EF_SOx = EF.iloc[:,1:-10][EF["Pollutant"]=="SOx"]
EF_PM = EF.iloc[:,1:-10][EF["Pollutant"]=="PM Exhaust"]

In [114]:
dfm = df_2.merge(EF_CO, left_on=["Región", "Ámbito", "Categoría", "Motorizacion", "Norma"], right_on=["Region", "Ambito", "Modo", "Motorizacion", "Norma"], how="left")
dfm=dfm.fillna(0)
df_CO = pd.DataFrame(dfm.iloc[:,6:43].values*dfm.loc[:,"2014_y":2050].values, columns=dfm.iloc[:,47:84].columns, index=dfm.index)
df_CO = pd.concat([dfm.iloc[:,:6],df_CO],axis=1)
df_CO["Unidad"]="[gr/año]"
df_CO
EM_CO = merger(df_2,EF_CO)
EM_CH4 = merger(df_2,EF_CH4)
EM_NOx = merger(df_2,EF_NOx)
EM_CO2 = merger(df_2,EF_CO2)
EM_SOx = merger(df_2,EF_SOx)
EM_PM = merger(df_2,EF_PM)

In [113]:
EM_CO2.to_excel("EM_CO2.xlsx")